In [100]:
import numpy as np
import random
import pandas as pd 
from copy import deepcopy
import math
import time
from himmelblau import *

In [101]:
class Individual:
    global_best_position=None
    global_best_fitness=None
    def __init__(self,bounds,velocity_calc,calc_fitness):
        self.lower_bounds = np.array([bound[0] for bound in bounds])
        self.upper_bounds = np.array([bound[1] for bound in bounds])        
        
        self.position = np.random.uniform(self.lower_bounds, self.upper_bounds, len(bounds))
        self.velocity = np.random.uniform(self.lower_bounds - self.upper_bounds,
                                          self.upper_bounds - self.lower_bounds,
                                          len(bounds))
        self.calc_fitness=calc_fitness
        self.fitness=calc_fitness(self.position)
        
        self.velocity_calc=velocity_calc
        
        self.personal_best_fitness=self.fitness
        self.personal_best_position=self.position.copy()
        
        if Individual.global_best_fitness is None or Individual.global_best_fitness<self.fitness:
                Individual.global_best_fitness=self.fitness
                Individual.global_best_position=self.position.copy()
            
    def update_position(self):
        self.position += self.velocity
        self.clip()
        self.update_fitness()
       
    def update_velocity(self):
        personal_direction=(self.personal_best_position - self.position)
        global_direction=(Individual.global_best_position - self.position)
        
        cognitive_velocity =random.random() * self.velocity_calc["cognitive"]*personal_direction 
        social_velocity =random.random() * self.velocity_calc["group"]* global_direction
        inertia = self.velocity_calc["inertia"] * self.velocity
        self.velocity = inertia + cognitive_velocity + social_velocity
        self.velocity=np.clip(self.velocity,self.lower_bounds,self.upper_bounds)
    def update_fitness(self):
        self.fitness=self.calc_fitness(self.position)
        if self.personal_best_fitness is None or self.fitness>self.personal_best_fitness:
            self.personal_best_fitness=self.fitness
            self.personal_best_position=self.position.copy()
            if Individual.global_best_fitness is None or Individual.global_best_fitness<self.fitness:
                Individual.global_best_fitness=self.fitness
                Individual.global_best_position=self.position.copy()
    
    def clip(self):
        self.position=np.clip(self.position,self.lower_bounds,self.upper_bounds)
    def mutate(self,mut_prob):
        for i in range(len(self.position)):
           if random.random()<mut_prob:
               self.position+=(-1)**(random.randrange(1,3))*np.random.uniform()
        self.clip()
        

In [102]:
def crossover(p1:Individual,p2:Individual,c1:Individual,c2:Individual):
    coef=random.random()
    c1.position=coef*p1.position + (1-coef)*p2.position
    c1.velocity=coef*p1.velocity + (1-coef)*p2.velocity
    
    c2.position=coef*p2.position +(1-coef)*p1.position
    c2.velocity=coef*p2.velocity +(1-coef)*p1.velocity

    

In [103]:
def mutation(ind:Individual,mut_prob=0.01):
    ind.mutate(mut_prob)

In [104]:
def selection(population,tournament_size):
    allowed=random.sample(population,tournament_size)
    return max(allowed,key=lambda x:x.fitness)

In [118]:
vezbe_velocity_calc={
    "cognitive":0.7 ,
    "group":1.0 ,
    "inertia":1.0
}
def psogahybrid(pop_size, num_dimensions, allowed_time,pso_to_ga_ratio,velocity_calc,fitness_calc,elitism_size,tournament_size=4):
    bounds = [(78, 102),(33,45),(27,45),(27,45),(27,45)]
    
    population = [Individual(bounds=bounds,
                      calc_fitness=fitness_calc,velocity_calc=velocity_calc
                      ) for i in range(pop_size)]
    #print(Individual.global_best_fitness)
    
    start=time.time()
    while True:
        
        if random.random()< pso_to_ga_ratio:
            
            for i in range(len(population)):
               
                population[i].update_velocity()
                population[i].update_position()
        else:
            sorted(population,key=lambda x:x.fitness,reverse=True)
            new_population=population[:]
            for i in range(elitism_size,len(population),2):            
                p1=selection(population,tournament_size)
                p2=selection(population,tournament_size)
                crossover(p1,p2,new_population[i],new_population[i+1])
                mutation(new_population[i])
                mutation(new_population[i+1])
                new_population[i].update_fitness()
                new_population[i+1].update_fitness()
            population=new_population           
        if time.time()-start>allowed_time:
                tmp=Individual.global_best_position
                if not is_feasable(Individual.global_best_position):
                    print("not feasible!")
                Individual.global_best_fitness=None
                Individual.global_best_position=None
                return actual_himmelblau(tmp)

In [147]:
#nasa hibridna optimizacija
psogahybrid(pop_size=50, num_dimensions=1, allowed_time=60,velocity_calc=vezbe_velocity_calc,pso_to_ga_ratio=0.85,elitism_size=4,fitness_calc=himmelblau_fittnes,tournament_size=15)

28532.757769078926

In [ ]:
#goal driven generational learning

Ideja algoritma:
    Imamo populaciju jedinki koje imaju svoju poziciju u prostoru i svoje usmerenje
    Imaju age-broj iteracija koliko je jedinka ziva.
    Imaju life_expectancy-zivotni vek u broju iteracija
    Pusticemo svaku jedinku da se krece kroz prostor nalik na pso dok ne bude imala 
    life expectancy godina.
    Razlika u odnosu na pso je sto dinamicki racunamo koeficijente prema broju godina i
    zivotnog ocekivanja.
    Kada jedinka dostigne life expectancy godina koristimo tournament selection da bi odabrali jedinku
    sa kojom cemo da je ukrstimo kako bi je zamenila u populaciji.(od dva deteta biramo najbolje)
    to radimo dok ne prodjemo maximalni broj iteracija.
    Mozda bi mutacija dovela do prevelikog diverziteta.
    Mozda bi neki elitism doprineo.
    

In [ ]:
times=[10**(-3),10**(-2),10**(-1),1]
num_trys=1000

In [ ]:
print("Function : Shaffer")
print("Population size: " + "50")
print("Num dimensions: " + "2")
print("c_i: "+"0.7")
print("c_p: " + "1.0")
print("c_g: " + "1.0")
print("Life expectancy: " + "50")
for allowed_time in times:
    suma=0
    for _ in range(num_trys):
        suma+=psogahybrid(pop_size=50, num_dimensions=2, allowed_time=allowed_time,velocity_calc=vezbe_velocity_calc,fitness_calc=shaffer,tournament_size=10)
    suma=-suma
    suma/=num_trys
    
    print("    Time : " + str(allowed_time))
    
    print("    Average mistake: " +str(suma))
    print("\n")